# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import List, Tuple, Dict, Any, Callable, Optional
import os
from datetime import datetime
import pandas as pd

In [ ]:
#| export

class DebugLogger:
    """Simple debug logger to centralize logging logic."""
    
    @staticmethod
    def log(message: str) -> None:
        """Log a debug message if DEBUG environment variable is TRUE."""
        if os.environ.get('DEBUG', 'FALSE').upper() == 'TRUE':
            print(message)

In [ ]:
#| export

class ColumnUtils:
    """Utilities for column operations in DataFrames."""
    
    @staticmethod
    def normalize_columns(df_columns: List[str]) -> Dict[str, str]:
        """Create a case-insensitive mapping of column names.
        
        Args:
            df_columns: List of DataFrame column names
            
        Returns:
            Dictionary mapping uppercase column names to their original case
        """
        return {col.upper(): col for col in df_columns}
    
    @staticmethod
    def get_column_synonyms() -> Dict[str, set]:
        """Get sets of column synonyms for common fields.
        
        Returns:
            Dictionary mapping field types to sets of possible column names
        """
        return {
            'copper_id': {"COMPANY_ID", "COPPER_ID"},
            'title':    {"NAME", "TITLE"}
        }
    
    @staticmethod
    def get_detail_columns(df_columns: List[str]) -> List[str]:
        """Filter columns to show in alert details.
        
        Args:
            df_columns: List of DataFrame columns
            
        Returns:
            Filtered columns for details
        """
        excluded_columns = [
            'COMPANY_ID', 'COPPER_ID', 'NAME',
            'TITLE', 'TITLE_LINK', 'OPTION_NAME', 'OPTION_VALUE','CONFIG','TK_CONFIG'
        ]
        
        return [
            column for column in df_columns 
            if not column.endswith('_h') 
            and not column.lower().endswith('_hide') 
            and not column.lower().endswith('_meta') 
            and not (column in excluded_columns)
        ]

In [ ]:
#| export


class ValueFormatter:
    """Handles formatting of different data types for display."""
    
    @staticmethod
    def format_value(val: Any) -> str:
        """Format a value for display based on its type.
        
        Args:
            val: Any value to format
            
        Returns:
            Formatted string representation of the value
        """
        #DebugLogger.log(f'val: {val}')
        
        # Handle null values
        if pd.isnull(val):
            #DebugLogger.log(f'Formatted: {str(val)}')
            return ""
            
        # Handle datetime objects
        if isinstance(val, (datetime, pd.Timestamp)):
            formatted = val.strftime('%b %d, %Y')
            #DebugLogger.log(f'Formatted: {formatted}')
            return formatted
            
        # Handle numeric values
        if isinstance(val, float):
            formatted = f"{val:.2f}"
            #DebugLogger.log(f'Formatted: {formatted}')
            return formatted
            
        if isinstance(val, int):
            formatted = str(val)
            #DebugLogger.log(f'Formatted: {formatted}')
            return formatted
            
        # Handle boolean values
        if isinstance(val, bool):
            formatted = "Yes" if val else "No"
            #DebugLogger.log(f'Formatted: {formatted}')
            return formatted
            
        # Handle collections
        if isinstance(val, (list, tuple)):
            return ", ".join(str(v) for v in val)
            
        if isinstance(val, dict):
            formatted = ", ".join(f"{k}: {v}" for k, v in val.items())
            #DebugLogger.log(f'Formatted: {formatted}')
            return formatted
            
        # Try to parse strings as dates, fallback to string
        if isinstance(val, str):
            try:
                parsed_date = pd.to_datetime(val)
                if isinstance(parsed_date, pd.Timestamp):
                    DebugLogger.log(f'val: {val}')
                    if parsed_date.year >= 2023:
                        formatted = parsed_date.strftime('%b %d, %Y')
                        #DebugLogger.log(f"Year greater than 2023: {formatted}")
                        return formatted
                    else:
                        DebugLogger.log(f"Year less than 2023: {parsed_date.year}")
            except Exception:
                pass
        
        # Default string formatting
        formatted = str(val)
        #DebugLogger.log(f'Formatted: {formatted}')
        return formatted


In [ ]:
#| export

class SlackFormatter:
    """Utilities for formatting data for Slack messages."""
    
    @staticmethod
    def right_hand_details(row: pd.Series, detail_columns: List[str], df: pd.DataFrame) -> str:
        """Format row details for Slack message with aligned values.
        
        Args:
            row: Row data
            detail_columns: Columns to include in details
            df: Full DataFrame
            
        Returns:
            Formatted detail text with aligned values
        """
        labels = [column.lower().replace('_', ' ').title() for column in detail_columns]
        max_label_length = max((len(label) for label in labels), default=0)

        row_details = [
            f"{label.ljust(max_label_length)}: ‎ {ValueFormatter.format_value(df.loc[row.name, column])}"
            for label, column in zip(labels, detail_columns)
        ]

        return '\n'.join(row_details)
    
    @staticmethod
    def format_section_name(row: pd.Series, df_columns: List[str]) -> str:
        """Create a Slack-formatted section title with optional Copper or custom link.

        Handles:
        - Synonyms for Copper ID: COMPANY_ID, COPPER_ID
        - Synonyms for Name: NAME, TITLE
        - Case-insensitive column matching
        - Cleaning of string and numeric types
        - Basic validation of TITLE_LINK

        Args:
            row: Row of data
            df_columns: List of column names in the DataFrame

        Returns:
            Slack-formatted string for the section title
        """
        # Use utility functions for column handling
        col_map = ColumnUtils.normalize_columns(df_columns)
        row_keys_upper = {k.upper(): k for k in row.index}
        synonyms = ColumnUtils.get_column_synonyms()

        copper_id = None
        title = None

        # Find copper ID from synonyms
        for key in synonyms['copper_id']:
            actual_key = row_keys_upper.get(key)
            if actual_key:
                val = row.get(actual_key)
                if isinstance(val, (int, float)) and not pd.isnull(val):
                    copper_id = int(val)
                    break
                # If string that looks like int
                elif isinstance(val, str) and val.strip().isdigit():
                    copper_id = int(val.strip())
                    break

        # Find title from synonyms
        for key in synonyms['title']:
            actual_key = row_keys_upper.get(key)
            if actual_key:
                val = row.get(actual_key)
                if isinstance(val, str):
                    val_clean = val.strip()
                    if val_clean and not any(char.isdigit() for char in val_clean):
                        title = val_clean
                        break

        # Fallback title
        if not title:
            title = "Untitled"

        # Copper ID takes priority
        if copper_id is not None:
            copper_url = f"https://app.copper.com/companies/326784/app#/browse/list/companies/default?fullProfile=companies-{copper_id}"
            return f"*<{copper_url}|{title}>*"

        # Check for valid TITLE_LINK
        title_link_key = row_keys_upper.get("TITLE_LINK")
        if title_link_key:
            title_link = row.get(title_link_key)
            if isinstance(title_link, str) and title_link.strip().lower().startswith("http"):
                return f"*<{title_link.strip()}|{title}>*"

        # Plain fallback
        return f"*{title}*"

In [ ]:
#| export

class SlackMessenger:
    """Handles creation and sending of Slack messages in various templates."""
    
    @staticmethod
    def process_section_row(section_text: str, detail_text: str) -> Dict[str, Any]:
        """Create Slack message section block.
        
        Args:
            section_text: Main section text with markdown formatting
            detail_text: Detailed information with markdown formatting
            
        Returns:
            Slack block kit section object
        """
        return {
            "type": "section",
            "fields": [
                {"type": "mrkdwn", "text": section_text},
                {"type": "mrkdwn", "text": detail_text}
            ]
        }
        
    @staticmethod
    def create_header_block(title: str) -> Dict[str, Any]:
        """Create a header block for Slack messages.
        
        Args:
            title: The title text for the header
            
        Returns:
            Slack block kit header object
        """
        return {
            "type": "header",
            "text": {"type": "plain_text", "text": title, "emoji": True}
        }
    
    @staticmethod
    def get_metadata(row: pd.Series, df_columns: List[str]) -> Dict[str, Any]:
        """Extract metadata from row for logging.
        
        Args:
            row: DataFrame row
            df_columns: List of column names
            
        Returns:
            Dictionary of column values
        """
        # Fixed the incorrect implementation
        return {column: row.get(column) for column in df_columns}
    
    @staticmethod
    def _send_alert_to_slack(
        send_to_slack_func: callable,
        channel_id: str,
        message_text: str,
        payload_blocks: List[Dict[str, Any]],
        view: str
    ) -> Tuple[bool, Optional[Dict[str, Any]]]:
        """Send alert to Slack and handle response.
        
        Args:
            send_to_slack_func: Function to send messages to Slack
            channel_id: Slack channel ID
            message_text: Main message text
            payload_blocks: Slack blocks to send
            view: View name for logging
            
        Returns:
            Tuple of (success_flag, error_details)
        """
        try:
            response = send_to_slack_func(channel_id, message_text, payload_blocks=payload_blocks)
            DebugLogger.log(f'Response: {response.text if hasattr(response, "text") else response}')
            
            # Check response
            if hasattr(response, 'status_code') and response.status_code == 200:
                # Handle JSON response
                if hasattr(response, 'json'):
                    try:
                        json_response = response.json()
                        if json_response.get('ok'):
                            print(f'   Alert sent for {view}')
                            return True, None
                    except Exception as json_err:
                        print(f'   Error parsing JSON response for {view}: {json_err}')
                        return False, {'slack_api_error': f'JSON parse error: {str(json_err)}'}
                else:
                    # Simple success response
                    print(f'   Alert sent for {view}')
                    return True, None
            
            # Handle error response
            error_text = response.text if hasattr(response, 'text') else str(response)
            print(f'   Error sending alert for {view}: {error_text}')
            return False, {'slack_api_error': error_text}
            
        except Exception as e:
            print(f'   Error sending alert for {view}: {e}')
            return False, {'slack_api_error': str(e)}
    
    @staticmethod
    def _format_data_for_logging(df: pd.DataFrame) -> List[Dict[str, Any]]:
        """Format DataFrame data for logging.
        
        Args:
            df: DataFrame with alert data
            
        Returns:
            List of dictionaries with formatted values
        """
        return [{
            key: ValueFormatter.format_value(value) 
            for key, value in row.items()
        } for row in df.to_dict('records')]
    
    @staticmethod
    def _log_alert(
        log_alert_history_func: callable,
        view: str,
        view_group: str,
        channel_id: str,
        success: bool,
        error_details: Optional[Dict[str, Any]],
        formatted_data: List[Dict[str, Any]],
        message_text: str
    ) -> None:
        """Log alert history.
        
        Args:
            log_alert_history_func: Function to log alert history
            view: View name
            view_group: Group name for the view
            channel_id: Slack channel ID
            success: Whether the alert was sent successfully
            error_details: Details of any error
            formatted_data: Formatted data for logging
            message_text: Main message text
        """
        try:
            log_alert_history_func(
                related_view=view,
                team=view_group,
                channel=channel_id,
                rows_grouped=True,
                row_num=None,
                data=formatted_data,
                was_success=success,
                error_details=error_details,
                message_details={'format': 'f1', 'message_text': message_text}
            )
        except Exception as e:
            print(f"Error logging alert history: {e}")


In [ ]:
#| export

class SlackMessenger:
    """Handles creation and sending of Slack messages in various templates."""
    
    @staticmethod
    def process_section_row(section_text: str, detail_text: str) -> Dict[str, Any]:
        """Create Slack message section block.
        
        Args:
            section_text: Main section text with markdown formatting
            detail_text: Detailed information with markdown formatting
            
        Returns:
            Slack block kit section object
        """
        return {
            "type": "section",
            "fields": [
                {"type": "mrkdwn", "text": section_text},
                {"type": "mrkdwn", "text": detail_text}
            ]
        }
        
    @staticmethod
    def create_header_block(title: str) -> Dict[str, Any]:
        """Create a header block for Slack messages.
        
        Args:
            title: The title text for the header
            
        Returns:
            Slack block kit header object
        """
        return {
            "type": "header",
            "text": {"type": "plain_text", "text": title, "emoji": True}
        }
    
    @staticmethod
    def get_metadata(row: pd.Series, df_columns: List[str]) -> Dict[str, Any]:
        """Extract metadata from row for logging.
        
        Args:
            row: DataFrame row
            df_columns: List of column names
            
        Returns:
            Dictionary of column values
        """
        # Fixed the incorrect implementation
        return {column: row.get(column) for column in df_columns}
    
    @staticmethod
    def _send_alert_to_slack(
        send_to_slack_func: callable,
        channel_id: str,
        message_text: str,
        payload_blocks: List[Dict[str, Any]],
        view: str
    ) -> Tuple[bool, Optional[Dict[str, Any]]]:
        """Send alert to Slack and handle response.
        
        Args:
            send_to_slack_func: Function to send messages to Slack
            channel_id: Slack channel ID
            message_text: Main message text
            payload_blocks: Slack blocks to send
            view: View name for logging
            
        Returns:
            Tuple of (success_flag, error_details)
        """
        try:
            response = send_to_slack_func(channel_id, message_text, payload_blocks=payload_blocks)
            DebugLogger.log(f'Response: {response.text if hasattr(response, "text") else response}')
            
            # Check response
            if hasattr(response, 'status_code') and response.status_code == 200:
                # Handle JSON response
                if hasattr(response, 'json'):
                    try:
                        json_response = response.json()
                        if json_response.get('ok'):
                            print(f'   Alert sent for {view}')
                            return True, None
                    except Exception as json_err:
                        print(f'   Error parsing JSON response for {view}: {json_err}')
                        return False, {'slack_api_error': f'JSON parse error: {str(json_err)}'}
                else:
                    # Simple success response
                    print(f'   Alert sent for {view}')
                    return True, None
            
            # Handle error response
            error_text = response.text if hasattr(response, 'text') else str(response)
            print(f'   Error sending alert for {view}: {error_text}')
            return False, {'slack_api_error': error_text}
            
        except Exception as e:
            print(f'   Error sending alert for {view}: {e}')
            return False, {'slack_api_error': str(e)}
    
    @staticmethod
    def _format_data_for_logging(df: pd.DataFrame) -> List[Dict[str, Any]]:
        """Format DataFrame data for logging.
        
        Args:
            df: DataFrame with alert data
            
        Returns:
            List of dictionaries with formatted values
        """
        return [{
            key: ValueFormatter.format_value(value) 
            for key, value in row.items()
        } for row in df.to_dict('records')]
    
    @staticmethod
    def _log_alert(
        log_alert_history_func: callable,
        view: str,
        view_group: str,
        channel_id: str,
        success: bool,
        error_details: Optional[Dict[str, Any]],
        formatted_data: List[Dict[str, Any]],
        message_text: str
    ) -> None:
        """Log alert history.
        
        Args:
            log_alert_history_func: Function to log alert history
            view: View name
            view_group: Group name for the view
            channel_id: Slack channel ID
            success: Whether the alert was sent successfully
            error_details: Details of any error
            formatted_data: Formatted data for logging
            message_text: Main message text
        """
        try:
            log_alert_history_func(
                related_view=view,
                team=view_group,
                channel=channel_id,
                rows_grouped=True,
                row_num=None,
                data=formatted_data,
                was_success=success,
                error_details=error_details,
                message_details={'format': 'f1', 'message_text': message_text}
            )
        except Exception as e:
            print(f"Error logging alert history: {e}")


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()